<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/hypersonic_multi_reactions_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Hypersonic Combustion with Multiple Reactions (Reduced GRI-Mech)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, FloatSlider, IntSlider

def run_sim(nx=200, dt=5e-7, t_end=0.001, mu=1e-5, k=0.02):
    gamma = 1.4
    R = 287.05
    L = 1.0
    dx = L / nx
    nt = int(t_end / dt)
    x = np.linspace(0, L, nx)

    # Species: H2, O2, H2O, CO, CO2, OH
    MW = {'H2':2.016, 'O2':32.00, 'H2O':18.02, 'CO':28.01, 'CO2':44.01, 'OH':17.01}
    Y = {s: np.zeros(nx) for s in MW}
    Y['H2'][:] = 0.05
    Y['O2'][:] = 0.21
    Y['CO'][:] = 0.01

    rho = np.ones(nx)
    u = np.ones(nx) * 900
    p = np.ones(nx) * 101325
    T = p / (rho * R)
    E = p / (gamma - 1) + 0.5 * rho * u**2

    A = 1 + 0.2 * np.cos(2 * np.pi * x / L)

    reactions = [
        {"r": ['H2', 'O2'], "p": ['OH', 'OH'], "A": 1e10, "Ea": 6e4, "dH": -5e6},
        {"r": ['CO', 'OH'], "p": ['CO2', 'H2O'], "A": 2e9, "Ea": 3e4, "dH": -4e6},
        {"r": ['H2', '0.5O2'], "p": ['H2O'], "A": 1e8, "Ea": 5e4, "dH": -2.4e6},
        {"r": ['CO', '0.5O2'], "p": ['CO2'], "A": 1e8, "Ea": 2.8e4, "dH": -2.8e6}
    ]

    snapshots = []

    def compute_diff(u, T):
        du_dx = np.gradient(u, dx)
        dT_dx = np.gradient(T, dx)
        tau = mu * du_dx
        qx = -k * dT_dx
        return tau, qx

    for step in range(nt):
        T = p / (rho * R)
        omega_total = np.zeros(nx)
        Q_total = np.zeros(nx)
        dY = {s: np.zeros(nx) for s in MW}

        # Process all reactions
        for rxn in reactions:
            A_rxn = rxn["A"]
            Ea = rxn["Ea"]
            dH = rxn["dH"]

            r1, *r_extra = rxn["r"]
            Y_r1 = Y[r1] if r1 in Y else np.zeros(nx)
            if '0.5O2' in r_extra:
                Y_r2 = np.sqrt(Y['O2'])
            else:
                r2 = r_extra[0] if r_extra else 'O2'
                Y_r2 = Y[r2] if r2 in Y else np.zeros(nx)

            omega = A_rxn * np.exp(-Ea / (R * T)) * Y_r1 * Y_r2
            omega_total += omega
            Q_total += omega * dH * dt

            # Update mass fractions (very simplified stoichiometry)
            dY[r1] -= omega * dt
            dY[r2] -= omega * dt if isinstance(r2, str) else 0
            for prod in rxn["p"]:
                dY[prod] += omega * dt

        # Update fields
        F1 = rho * u
        F2 = rho * u**2 + p
        F3 = (E + p) * u
        tau, qx = compute_diff(u, T)

        rho_new, u_new, E_new = rho.copy(), u.copy(), E.copy()
        for i in range(1, nx - 1):
            rho_new[i] -= dt / dx * (F1[i] - F1[i - 1])
            mom = rho[i]*u[i] - dt / dx * (F2[i] - F2[i - 1]) + dt / dx * (tau[i] - tau[i - 1])
            if rho_new[i] > 1e-5:
                u_new[i] = mom / rho_new[i]
            E_flux = (F3[i] - F3[i - 1]) - (tau[i]*u[i] - tau[i-1]*u[i-1]) - (qx[i] - qx[i - 1])
            E_new[i] -= dt / dx * E_flux
            E_new[i] += Q_total[i]

        for s in Y:
            Y[s] += dY[s]
            Y[s] = np.clip(Y[s], 0, 1)

        p_new = (E_new - 0.5 * rho_new * u_new**2) * (gamma - 1)
        p_new = np.maximum(p_new, 1e-2)

        rho, u, E, p = rho_new, u_new, E_new, p_new

        if step % 100 == 0:
            snapshots.append((rho.copy(), u.copy(), p.copy(), T.copy(), {k: Y[k].copy() for k in Y}))

    # Export final state
    final = { "x": x, "rho": rho, "u": u, "p": p, "T": p / (rho * R) }
    for s in Y:
        final[f"Y_{s}"] = Y[s]
    df = pd.DataFrame(final)
    df.to_csv("multi_species_reactive_results.csv", index=False)
    print("✅ Exported to multi_species_reactive_results.csv")

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    line_u, = ax.plot(x, snapshots[0][1], label='u (m/s)')
    line_T, = ax.plot(x, snapshots[0][3], label='T (K)')
    line_YH2, = ax.plot(x, snapshots[0][4]['H2'], label='Y_H2')
    ax.set_ylim(0, 3000)
    ax.set_title("Multiple-Reaction Combustion")
    ax.legend()

    def animate(i):
        _, u_i, _, T_i, Ys = snapshots[i]
        line_u.set_ydata(u_i)
        line_T.set_ydata(T_i)
        line_YH2.set_ydata(Ys['H2'])
        ax.set_title(f"Step {i*100}")
        return line_u, line_T, line_YH2

    ani = FuncAnimation(fig, animate, frames=len(snapshots), interval=50)
    plt.show()

interact(run_sim,
         nx=IntSlider(min=100, max=400, step=10, value=200, description='nx'),
         dt=FloatSlider(min=1e-7, max=1e-5, step=1e-7, value=5e-7, description='Δt'),
         t_end=FloatSlider(min=0.0005, max=0.01, step=0.0005, value=0.001, description='t_end'),
         mu=FloatSlider(min=1e-6, max=1e-4, step=1e-6, value=1e-5, description='Visc μ'),
         k=FloatSlider(min=0.001, max=0.1, step=0.001, value=0.02, description='Thermal k'));


interactive(children=(IntSlider(value=200, description='nx', max=400, min=100, step=10), FloatSlider(value=5e-…